In [6]:
import numpy
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import mediapipe as mp
from tkinter import Tk, filedialog

In [7]:
model = hub.load("./mobilenet-v2-tensorflow1-openimages-v4-ssd-mobilenet-v2-v1").signatures["default"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [8]:
import random
import math

In [9]:
colorcodes = {}

In [10]:
def drawbox(image,ymin,xmin,ymax,xmax,namewithscore,color):
    im_height, im_width, _  = image.shape
    left,top,right,bottom = int(xmin*im_width), int(ymin*im_height), int(xmax*im_width),int(ymax*im_height)
    cv2.rectangle(image,(left,top),(right,bottom),color = color,thickness = 4)
    FONT_SCALE = 5e-3
    THICKNESS_SCALE = 4e-3
    width = right-left
    height = bottom-top
    TEXT_Y_OFFSET_SCALE = 1e-2
    cv2.rectangle(
        image,
        (left,top- int(height * 6e-2)),
        (right,top),
        color = color,
        thickness = -1
        
    )
    cv2.putText(
        image,
        namewithscore,
        (left,top-int(height * TEXT_Y_OFFSET_SCALE)),
        fontFace = cv2.FONT_HERSHEY_PLAIN,
        fontScale = min(width,height)* FONT_SCALE,
        thickness = math.ceil(min(width,height)* THICKNESS_SCALE),
        color = (255,255,255)
    )

In [11]:
def draw(image,boxes,classnames,scores):
    boxesidx = tf.image.non_max_suppression(boxes,scores,max_output_size = 4, iou_threshold = 0.5,score_threshold = 0.1)
#     for i in range(len(boxes)):
    for i in boxesidx:
        ymin,xmin,ymax,xmax = tuple(boxes[i])
        classname = classnames[i].decode("ascii")
        if classname in colorcodes.keys():
            color = colorcodes[classname]
        else:
            c1 = random.randrange(0,255,30)
            c2 = random.randrange(0,255,25)
            c3 = random.randrange(0,255,50)
            colorcodes.update({classname: (c1,c2,c3)})
            color = colorcodes[classname]
        namewithscore = "{}:{}".format(classname,int(100*scores[i]))
        drawbox(image,ymin,xmin,ymax,xmax,namewithscore,color)
        
    return image

In [13]:
Tk().withdraw()

# Open a file dialog to select an image
file_path = filedialog.askopenfilename(
    title="Select an Image",
    filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")]
)

if file_path:
    image = cv2.imread(file_path)
    image = cv2.resize(image, (800, 600))
    image2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    converted_img = tf.image.convert_image_dtype(image2, tf.float32)[tf.newaxis, ...]
    print("Image loaded and processed successfully.")
else:
    print("No file selected.")

detection=model(converted_img)

print(detection["detection_class_entities"])


result={key:value.numpy() for key,value in detection.items()}
imagewithboxes=draw(image,result["detection_boxes"],result["detection_class_entities"],result[  "detection_scores"])
cv2.imshow("image",imagewithboxes)
cv2.waitKey(0)

Image loaded and processed successfully.
tf.Tensor(
[b'Human face' b'Man' b'Clothing' b'Human eye' b'Human nose' b'Human hair'
 b'Human eye' b'Shirt' b'Human ear' b'Human eye' b'Human arm'
 b'Human beard' b'Human eye' b'Human nose' b'Human hair' b'Human eye'
 b'Shirt' b'Human eye' b'Human ear' b'Human head' b'Human eye'
 b'Human mouth' b'Human hair' b'Human mouth' b'Human eye' b'Human arm'
 b'Human hair' b'Human ear' b'Person' b'Human hair' b'Human hair'
 b'Human eye' b'Human nose' b'Shirt' b'Human nose' b'Human mouth'
 b'Human hair' b'Shirt' b'Human nose' b'Human ear' b'Human hair'
 b'Human hair' b'Human hair' b'Human hair' b'Shirt' b'Man' b'Human nose'
 b'Human nose' b'Human mouth' b'Human hair' b'Clothing' b'Human hair'
 b'Human eye' b'Human hair' b'Human hair' b'Shirt' b'Human eye'
 b'Human hair' b'Human hair' b'Clothing' b'Fashion accessory' b'Human arm'
 b'Human hair' b'Human hair' b'Human beard' b'Human nose' b'Human hair'
 b'Human hair' b'Human hair' b'Human eye' b'Human ear' b

-1